In [ ]:
! pip install xgboost

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBRegressor
import warnings
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/DataGenieHackathon/Sample Time Series/daily/results_daily.csv')
df.head(10)

,Mean,Variance,Skewness,Kurtosis,Trend Mean,Seasonal Mean,Residual Mean,Forecast_Score
0,6.383838,0.340961,1.232438,0.509600,6.371944,-0.007409,-0.010127,9.161971
1,6.486486,0.433907,1.008367,-0.145191,6.423611,-0.007449,-0.018374,9.078100
2,6.560976,0.461415,0.802197,-0.516651,6.513468,-0.008352,0.000434,9.061807
3,6.659259,0.539746,0.628536,-0.905793,6.607733,-0.004066,-0.010069,9.006383
4,6.721088,0.558662,0.500972,-1.055607,6.693767,-0.007368,0.002691,8.999218
5,6.779874,0.577820,0.390080,-1.171620,6.765432,-0.001876,0.000029,8.991853
6,6.824561,0.580805,0.305090,-1.216723,6.821145,-0.005732,0.000331,8.995467
7,6.868852,0.587101,0.226005,-1.259361,6.873428,-0.000159,-0.002563,8.996441
8,6.887179,0.564526,0.186995,-1.203007,6.911428,-0.003871,-0.000341,9.016369
9,6.917874,0.561184,0.134234,-1.201378,6.934426,0.000021,-0.002403,9.022926


In [ ]:
df.tail(10)

,Mean,Variance,Skewness,Kurtosis,Trend Mean,Seasonal Mean,Residual Mean,Forecast_Score
89,281330.077945,2.174977e+11,2.016856,4.073553,285131.473035,49.989532,38.220957,3.762623
90,280443.236690,2.154360e+11,2.030737,4.152432,284274.557794,4.727564,-131.939954,3.766391
91,280013.407507,2.133501e+11,2.042690,4.227751,283438.811105,47.495402,-172.511356,3.774215
92,279459.533523,2.113376e+11,2.054900,4.303151,282744.562991,4.007296,14.169193,3.780699
93,278679.520970,2.093912e+11,2.068115,4.380855,282153.840787,47.955701,60.492013,3.785006
94,278165.407504,2.074445e+11,2.080222,4.456507,281480.799281,2.860983,-124.102300,3.791651
95,277728.675911,2.055295e+11,2.091870,4.531233,280867.991625,43.611892,-15.094365,3.798870
96,277017.321112,2.036797e+11,2.104817,4.608996,280355.244150,5.092154,-9.757528,3.803479
97,276619.945018,2.018505e+11,2.115922,4.682023,279784.854457,44.639967,-139.049767,3.810729
98,276880.271482,2.000812e+11,2.122422,4.740648,279371.800512,1.466486,-192.632619,3.823338


In [ ]:
df.corr()

,Mean,Variance,Skewness,Kurtosis,Trend Mean,Seasonal Mean,Residual Mean,Forecast_Score
Mean,1.000000,0.986997,0.523823,0.098869,0.998770,0.368465,0.012405,-0.848436
Variance,0.986997,1.000000,0.548033,0.125421,0.980001,0.327239,0.012981,-0.869348
Skewness,0.523823,0.548033,1.000000,0.854122,0.492895,0.189135,-0.169209,-0.873840
Kurtosis,0.098869,0.125421,0.854122,1.000000,0.067139,0.039287,-0.188088,-0.558598
Trend Mean,0.998770,0.980001,0.492895,0.067139,1.000000,0.376544,0.019095,-0.825632
Seasonal Mean,0.368465,0.327239,0.189135,0.039287,0.376544,1.000000,-0.010099,-0.302405
Residual Mean,0.012405,0.012981,-0.169209,-0.188088,0.019095,-0.010099,1.000000,0.099054
Forecast_Score,-0.848436,-0.869348,-0.873840,-0.558598,-0.825632,-0.302405,0.099054,1.000000


In [ ]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [ ]:
path_map = {
    "daily" : "/content/drive/MyDrive/DataGenieHackathon/Sample Time Series/daily/results_daily.csv" ,
    "hourly" : "/content/drive/MyDrive/DataGenieHackathon/Sample Time Series/hourly/results_hourly.csv" ,
    "monthly" : "/content/drive/MyDrive/DataGenieHackathon/Sample Time Series/monthly/results_monthly.csv" ,
    "weekly" : "/content/drive/MyDrive/DataGenieHackathon/Sample Time Series/weekly/results_weekly.csv" ,
}

for ts,path in path_map.items():
  df = pd.read_csv(path , index_col = 0)
  # Split features and target variable
  X = df.drop('Forecast_Score', axis=1)
  y = df['Forecast_Score']

  # Split the data into training and testing sets
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Step 3: Train the XGBRegressor model
  xgb_model = XGBRegressor()
  xgb_model.fit(X_train, y_train)
  y_train_pred = xgb_model.predict(X_train)
  y_test_pred = xgb_model.predict(X_test)
  model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
  model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)
  print("*"*5+str(ts)+"*"*5)
  print('Model performance for Training set')
  print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
  print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
  print("- R2 Score: {:.4f}".format(model_train_r2))
  print('----------------------------------')
  print('Model performance for Test set')
  print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
  print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
  print("- R2 Score: {:.4f}".format(model_test_r2))
  model_path = '/content/drive/MyDrive/DataGenieHackathon/Sample Time Series/models/'+str(ts)+"model"+".pkl"
  # Save the trained model as a pickle file
  with open(model_path, 'wb') as model_file:
      pickle.dump(xgb_model, model_file)

*****daily*****
Model performance for Training set
- Root Mean Squared Error: 0.0009
- Mean Absolute Error: 0.0007
- R2 Score: 1.0000
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.0723
- Mean Absolute Error: 0.0391
- R2 Score: 0.9993
*****hourly*****
Model performance for Training set
- Root Mean Squared Error: 0.0007
- Mean Absolute Error: 0.0005
- R2 Score: 1.0000
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.0166
- Mean Absolute Error: 0.0148
- R2 Score: 0.9392
*****monthly*****
Model performance for Training set
- Root Mean Squared Error: 0.0007
- Mean Absolute Error: 0.0005
- R2 Score: 1.0000
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.0892
- Mean Absolute Error: 0.0573
- R2 Score: 0.9987
*****weekly*****
Model performance for Training set
- Root Mean Squared Error: 0.0007
- Mean Absolute Error: 0.0005
- R2 Score: 1.0000
-------------